In [5]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report


In [6]:
# Create the database connection
con = duckdb.connect(".config/nfl.duckdb")

In [ ]:
# NFL Teams ordered by their wins, descending
con.sql("""
    WITH game_results AS (
    SELECT
        homeTeamAbbr AS team,
        CASE
            WHEN homeFinalScore > visitorFinalScore THEN 1
            ELSE 0
        END AS win,
        CASE
            WHEN homeFinalScore < visitorFinalScore THEN 1
            ELSE 0
        END AS loss
    FROM bronze.games
    UNION ALL
    SELECT
        visitorTeamAbbr AS team,
        CASE
            WHEN visitorFinalScore > homeFinalScore THEN 1
            ELSE 0
        END AS win,
        CASE
            WHEN visitorFinalScore < homeFinalScore THEN 1
            ELSE 0
        END AS loss
    FROM bronze.games
    )
    SELECT
        team,
        SUM(win) AS total_wins,
        SUM(loss) AS total_losses
    FROM game_results
    GROUP BY team
    ORDER BY total_wins DESC;

""")

In [79]:
# Creating dataframes with DuckDB
train = con.sql("""
    SELECT *
    FROM bronze.tracking_week_1
    WHERE event != 'NA'
        AND o != 'NA'
        AND dir != 'NA'
        """).df()


In [ ]:
# Generating dummy variables with Pandas
train_one_hot = pd.get_dummies(train[["event"]], drop_first = True)
train_one_hot = pd.concat([train, train_one_hot], axis=1)
train_one_hot = train_one_hot.drop(["event"], axis = 1)
train_one_hot = train_one_hot.drop(['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'frameType',
       'time', 'jerseyNumber', 'playDirection', 'club'], axis = 1)
train_one_hot.head()

In [10]:
con.sql("SELECT DISTINCT play_outcome FROM bronze.agg_play_summary")

┌──────────────────┐
│   play_outcome   │
│     varchar      │
├──────────────────┤
│ Sacked           │
│ Rushing Play     │
│ Intercepted Pass │
│ Complete Pass    │
│ Incomplete Pass  │
└──────────────────┘

In [ ]:
# Need to use the predictions to attach the event labels to the dataset to plot
# Create scatter plot
plt.scatter(X_train, y_train, color="blue")

# Create line plot
plt.plot(X_train, predictions, color="red")
plt.xlabel("Play data ($)")
plt.ylabel("Event")

# Display the plot
plt.show()